## Named Netity recognition: NER conll2003
### Ildar Nurgaliev (Innopolis university)

In [1]:
!ls -l ./data/eng*

-rw-rw-r-- 1 ildar ildar  423703 Nov 16 16:05 ./data/eng.testa.conll
-rw-rw-r-- 1 ildar ildar  383154 Nov 16 16:05 ./data/eng.testb.conll
-rw-rw-r-- 1 ildar ildar 1683942 Nov 16 16:05 ./data/eng.train.conll


In [82]:
import gensim
from gensim.parsing.preprocessing import STOPWORDS
import itertools
import nltk
import os
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

get_full_path = lambda f_name : os.path.join(os.getcwd(), 'data', f_name)
fn_train = "eng.train.conll"
fn_testa = "eng.testa.conll"
fn_testb = "eng.testb.conll"
DELIM = '_'

### Analyse files

In [25]:
def show_file(f_name, n_from=0, n_row=10):
    fname = get_full_path(f_name)
    r = []
    with open(fname) as f:
        print("".join(itertools.islice(f.readlines(), n_from, n_from + n_row)))


show_file(fn_train, n_from=0, n_row=20)
# show_file(fn_testa, n_from=0, n_row=20)
# show_file(fn_testb, n_from=0, n_row=20)

O	-DOCSTART-
	
B-ORG	EU
O	rejects
B-MISC	German
O	call
O	to
O	boycott
B-MISC	British
O	lamb
O	.
	
B-PER	Peter
I-PER	Blackburn
	
B-LOC	BRUSSELS
O	1996-08-22
	
O	The
B-ORG	European

O	-DOCSTART-
	
O	CRICKET
O	-
B-ORG	LEICESTERSHIRE
O	TAKE
O	OVER
O	AT
O	TOP
O	AFTER
O	INNINGS
O	VICTORY
O	.
	
B-LOC	LONDON
O	1996-08-30
	
B-MISC	West
I-MISC	Indian
O	all-rounder

O	-DOCSTART-
	
O	SOCCER
O	-
B-LOC	JAPAN
O	GET
O	LUCKY
O	WIN
O	,
B-PER	CHINA
O	IN
O	SURPRISE
O	DEFEAT
O	.
	
B-PER	Nadim
I-PER	Ladki
	
B-LOC	AL-AIN
O	,



In [32]:
def show_form(f_name, size_show = 5, labels=None):
    fname = get_full_path(f_name)
    res = []
    with open(fname, 'r') as f:
        content = f.readlines()
    d = {}
    for line in content:
        line = line.strip() # remove newline from the end of the line
    
        if not len(line): #skip blank lines
            continue

        label, word = line.split("\t")
        if labels:
            if label in labels:
                if label in d:
                    d[label].append(word)
                else:
                   d[label] = [word] 
        else:
            if label in d:
                d[label].append(word)
            else:
               d[label] = [word]
    for v in d:
        print("%s[%d]\t%s" % (v, len(d[v]), " || ".join(d[v][:size_show])))
print("______________TRAIN")
show_form(fn_train, size_show = 5)
print("______________TEST_A")
show_form(fn_testa, size_show = 5)
print("______________TEST_B")
show_form(fn_testb, size_show = 5)

______________TRAIN
I-ORG[3704]	Commission || Union || Union || National || Farmers
B-MISC[3438]	German || British || German || British || EU-wide
I-PER[4528]	Blackburn || Zwingmann || van || der || Pas
I-LOC[1157]	Strait || Heights || East || States || Kurdistan
B-LOC[7140]	BRUSSELS || Germany || Britain || Britain || France
B-PER[6600]	Peter || Werner || Nikolaus || Franz || Fischler
I-MISC[1155]	Spongiform || Encephalopathy || n't || no || telling
O[170524]	-DOCSTART- || rejects || call || to || boycott
B-ORG[6321]	EU || European || European || Commission || European
______________TEST_A
I-ORG[751]	and || County || Cricket || Board || Universities
B-MISC[922]	West || Australian || ex-England || ENGLISH || English
I-PER[1307]	Simmons || Caddick || Hussain || Such || Lewis
I-LOC[257]	Road || Wells || Oval || 's || Oval
I-MISC[346]	Indian || COUNTY || CHAMPIONSHIP || Cup || 96
B-PER[1842]	Phil || Andy || Simmons || Nasser || Peter
B-LOC[1837]	LONDON || Grace || England || Headingley ||

In [2]:
def show_sentences(f_name):
    fname = get_full_path(f_name)
    rows = []
    with open(fname, 'r') as f:
        rows = f.readlines()
    sents = []
    sent = []
    for row in rows:
        row = row.strip()
        if row == '':
            if len(sent) > 0:
                yield ' '.join(sent)
            sent = []
            continue
        pos, word = row.split('\t',1)
        sent.append(word)
    print(f_name, len(sents), 'sentences')
frm = 100
for x in enumerate(itertools.islice(show_sentences(fn_train), frm, frm+10), frm):
    print(x)

(100, 'Port conditions from Lloyds Shipping Intelligence Service --')
(101, 'LATTAKIA , Aug 10 - waiting time at Lattakia and Tartous presently 24 hours .')
(102, '-DOCSTART-')
(103, 'Israel plays down fears of war with Syria .')
(104, 'Colleen Siegel')
(105, 'JERUSALEM 1996-08-22')
(106, "Israel 's outgoing peace negotiator with Syria said on Thursday current tensions between the two countries appeared to be a storm in a teacup .")
(107, "Itamar Rabinovich , who as Israel 's ambassador to Washington conducted unfruitful negotiations with Syria , told Israel Radio it looked like Damascus wanted to talk rather than fight .")
(108, '" It appears to me the Syrian priority is still to negotiate .')
(109, 'The Syrians are confused , they are definitely tense , but the general assessment here in Washington is that this is essentially a storm in a teacup , " he said .')


### Lemmatizator limited to (NN|VB|JJ|RB)

In [60]:
from gensim.parsing.preprocessing import STOPWORDS
from pattern.en import parse
from gensim.utils import tokenize
import re

def lemmatize(content, stopwords = STOPWORDS, allowed_tags=re.compile('(NN|VB|JJ|RB)')):
    content = " ".join(content)
#     content = " ".join(tokenize(content, lower=True, errors='ignore')) #ToDo replace tokenizator
    sentence = parse(content, lemmata=True, collapse=False)
    result = []
    for _, tag, _, _, lemma in sentence.pop():   
        if 2 <= len(lemma) <= 15 and not lemma.startswith('_') and lemma not in stopwords and allowed_tags.match(tag):
            lemma += DELIM + tag[:2]
            result.append(lemma.encode('utf8'))
        else:
            lemma += DELIM + "0"
            result.append(lemma.encode('utf8'))
    return result

### Sentence interator
yield: sentence as word_NER & sentence as lemma_POS

In [61]:
doc_counter = 0 # accept -DOCSTART- twice
def iter_sentences(f_name, lemma=False):
    fname = get_full_path(f_name)
    global doc_counter
    with open(fname, 'r') as rows:
        sent_poss = []
        sent = []
        for row in rows:
            row = row.strip()
            if row == '':
                if len(sent) > 0:
                    yield zip(sent_poss, lemmatize(sent))
                sent = []
                sent_poss = []
                continue
            pos, word = row.split('\t',1)
            if word != "-DOCSTART-":
                sent.append(word)
                sent_poss.append(word + DELIM + pos)
            elif doc_counter < 2:
                doc_counter += 1
                sent.append(word)
                sent_poss.append(word + DELIM + pos)                

iter_sen = itertools.islice(iter_sentences(fn_train, lemma=True), 2)
print next(iter_sen)
print next(iter_sen)

[('-DOCSTART-_O', '-_0')]
[('EU_B-ORG', 'eu_NN'), ('rejects_O', 'reject_VB'), ('German_B-MISC', 'german_JJ'), ('call_O', 'call_0'), ('to_O', 'to_0'), ('boycott_O', 'boycott_VB'), ('British_B-MISC', 'british_JJ'), ('lamb_O', 'lamb_NN'), ('._O', '._0')]


### Use any possible update
1. Generate and/or use Brown clusters
    - Software: http://dx.doi.org/10.5281/zenodo.33758
- Generate / use word embeddings Find from e.g. GLoVe site
- Use feature selection
- Create your own test dataset
    - Annotate some data that you found yourself
    
### We decided to cluster words by LDA and use these clusters for feature "cluster_num"
First we vectorize our data by representing each sentence as a 10k dimensional vector whose indices correspond to the 10k most frequent terms in our corpus. We then feed this N x 10,000 feature matrix into LDA to detect the latent topics in our data. While there are non-parametric alternatives available, the classic version of LDA allows us to specify the number of topics that we want to discover upfront. We run LDA for 2,000 iterations to identify 15 topics and receive a N x 15 matrix of topic distributions of our sentences.

In [62]:
doc_counter = 0
def iter_sentences_raw(f_name):
    global doc_counter
    fname = get_full_path(f_name)
    with open(fname, 'r') as rows:
        sent = []
        for row in rows:
            row = row.strip()
            if row == '':
                if len(sent) > 0:
                    yield ' '.join(sent)
                sent = []
                continue
            _, word = row.split('\t',1)
            if word != "-DOCSTART-":
                sent.append(word)
            elif doc_counter < 2:
                doc_counter += 1
                sent.append(word)
# train -> testa -> testb                
def chain_all_text():
    return itertools.chain(iter_sentences_raw(fn_train), iter_sentences_raw(fn_testa),iter_sentences_raw(fn_testb))
    
iter_sen = itertools.islice(chain_all_text(), 2)
print next(iter_sen)
print next(iter_sen)

-DOCSTART-
EU rejects German call to boycott British lamb .


### Check if sentences do not dissapear

In [5]:
get_sent_count = lambda fn : sum(1 for _ in iter_sentences_raw(fn))
print "raw"
print "TRAIN -", get_sent_count(fn_train)
print "TESTA -", get_sent_count(fn_testa)
print "TESTB -", get_sent_count(fn_testb)
print "Chain all text", sum(1 for _ in chain_all_text())
get_sent_count2 = lambda fn : sum(1 for _ in iter_sentences(fn))
print "tupled"
print "TRAIN -", get_sent_count2(fn_train)
print "TESTA -", get_sent_count2(fn_testa)
print "TESTB -", get_sent_count2(fn_testb)

raw
TRAIN - 14042
TESTA - 3250
TESTB - 3453
Chain all text 20744
tupled
TRAIN - 14041
TESTA - 3250
TESTB - 3453


### LDA implementation

In [5]:
%%time
import lda
from sklearn.feature_extraction.text import CountVectorizer

cvectorizer = CountVectorizer(min_df=4, max_features=10000, stop_words='english')
cvz = cvectorizer.fit_transform(chain_all_text())

n_topics = 15
n_iter = 2000
lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter)
X_topics = lda_model.fit_transform(cvz)

INFO : n_documents: 20745
INFO : vocab_size: 7207
INFO : n_words: 144178
INFO : n_topics: 15
INFO : n_iter: 2000
INFO : <0> log likelihood: -1863397
INFO : <10> log likelihood: -1293610
INFO : <20> log likelihood: -1231260
INFO : <30> log likelihood: -1205963
INFO : <40> log likelihood: -1191247
INFO : <50> log likelihood: -1182642
INFO : <60> log likelihood: -1176171
INFO : <70> log likelihood: -1171725
INFO : <80> log likelihood: -1168669
INFO : <90> log likelihood: -1165838
INFO : <100> log likelihood: -1164130
INFO : <110> log likelihood: -1163161
INFO : <120> log likelihood: -1161396
INFO : <130> log likelihood: -1160537
INFO : <140> log likelihood: -1159726
INFO : <150> log likelihood: -1158657
INFO : <160> log likelihood: -1158433
INFO : <170> log likelihood: -1157539
INFO : <180> log likelihood: -1157196
INFO : <190> log likelihood: -1156330
INFO : <200> log likelihood: -1156014
INFO : <210> log likelihood: -1155839
INFO : <220> log likelihood: -1155546
INFO : <230> log likelih

CPU times: user 1min 4s, sys: 344 ms, total: 1min 4s
Wall time: 1min 4s


In [83]:
doc_topic = lda_model.doc_topic_  #ToDo use it as feature for NER

inspect the words that are most relevant to a topic.

In [9]:
import numpy as np
n_top_words = 8
topic_summaries = []
t = 100
topic_word = lda_model.topic_word_  # get the topic words
vocab = cvectorizer.get_feature_names()
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: said government israel minister china president israeli foreign
Topic 1: said market percent company shares bank prices year
Topic 2: 70 69 71 72 67 13 66 68
Topic 3: minute minutes ahmed half win goal van barcelona
Topic 4: said president party told minister clinton government political
Topic 5: 1996 08 12 new 28 york 06 27
Topic 6: said police year old man told years people
Topic 7: said did think make like good new time
Topic 8: 10 15 21 22 16 11 17 20
Topic 9: 000 million percent year 30 20 10 tonnes
Topic 10: soccer division league results played cup standings saturday
Topic 11: said police people killed iraq group town rebels
Topic 12: round world second open women metres men tennis
Topic 13: new england day run innings pakistan cricket australia
Topic 14: beat 15 france germany italy 10 san south


### Dimension reduction

In [10]:
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_lda = tsne_model.fit_transform(X_topics[:10000])

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 10000
[t-SNE] Computed conditional probabilities for sample 2000 / 10000
[t-SNE] Computed conditional probabilities for sample 3000 / 10000
[t-SNE] Computed conditional probabilities for sample 4000 / 10000
[t-SNE] Computed conditional probabilities for sample 5000 / 10000
[t-SNE] Computed conditional probabilities for sample 6000 / 10000
[t-SNE] Computed conditional probabilities for sample 7000 / 10000
[t-SNE] Computed conditional probabilities for sample 8000 / 10000
[t-SNE] Computed conditional probabilities for sample 9000 / 10000
[t-SNE] Computed conditional probabilities for sample 10000 / 10000
[t-SNE] Mean sigma: 0.000000
[t-SNE] Error after 100 iterations with early exaggeration: 1.128578
[t-SNE] Error after 300 iterations: 1.049579


### doc_topic retrieve

In [11]:
lda_keys = []
count = 10000
for i, sent in enumerate(itertools.islice(iter_sentences_raw(fn_train), count)):
    lda_keys += [doc_topic[i].argmax()]

In [90]:
lda_keys[:20]

[11, 5, 4, 11, 11, 7, 11, 11, 11, 11, 0, 0, 11, 11, 2, 11, 10, 11, 13, 13]

### Visualize

In [12]:
%matplotlib inline
import bokeh.plotting as bp
from bokeh.io import push_notebook, show, output_notebook
import matplotlib.pyplot as plt
from bokeh.models import HoverTool
output_notebook()
colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c", 
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5", 
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f", 
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5"
])

plot_lda = bp.figure(plot_width=900, plot_height=700, title="Conll2003 (LDA)",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], 
                 color=colormap[lda_keys][:count], 
                 source=bp.ColumnDataSource({
                    "processed": list(itertools.islice(iter_sentences_raw(fn_train),10000)),
                    "topic_key": lda_keys[:count]
                })
                )
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips={"text": "(sent: \"@processed\" - topic: @topic_key)"}
show(plot_lda, notebook_handle=True)

Loading BokehJS ...

/home/ildar/miniconda3/envs/ds2/lib/python2.7/site-packages/bokeh/util/deprecation.py:33: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/home/ildar/miniconda3/envs/ds2/lib/python2.7/site-packages/bokeh/util/deprecation.py:33: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/home/ildar/miniconda3/envs/ds2/lib/python2.7/site-packages/bokeh/util/deprecation.py:33: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/home/ildar/miniconda3/envs/ds2/lib/python2.7/site-packages/bokeh/util/deprecation.py:33: BokehDeprecationWarning: 
Supplyin

### Feature extracture

In [101]:
# list of (word_ner, lexem_tag), pos
from nltk.corpus import words
words = set(words.words())
# feature functions
def c_shape(c):
    if c.isupper():
        c ='X'
    elif c.isalpha():
        c ='x'
    elif c.isdigit():
        c ='d'
    return c
w_shape = lambda w: ''.join(map(c_shape,w))
is_upper = lambda x : x[0].isupper()
is_digit = lambda x : x[0].isdigit()
is_word = lambda x : x in words
def num_same_elemseq(x):
        el = len(set(x))
        if el == 1:
            return "1"
        elif el == 2:
            return "2"
        elif el == 0:
            return "0"
        else:
            return "+"
def rev_type(x):
    if is_upper(x):
        return "1"
    elif is_digit(x):
        return "2"
    elif x.islower():
        return "3"
    else:
        return "0"
    
def retrieve(sent, iw):
    word_tup = sent[iw]
    try:
        word, _ = word_tup[0].rsplit(DELIM, 1 )
        lexem, pos = word_tup[1].rsplit(DELIM, 1)
        return (word, lexem, pos)
    except ValueError as v:
        print(word_tup)
        
def word_features(sent, iw, topic):
    word, lexem, pos = retrieve(sent, iw)    
#   word features
    features = [
        'w.len=%s' % len(word),
        'w.type=%s' % rev_type(word),
        'w.iw=%s' % iw,
        'w.shape=%s' % w_shape(word),       
        's.topic=%s' % topic,        
#         'w.lexem=%s' % lexem,
        'w.pos=%s' % pos,
        'w.is_wrd=%s' % is_word(lexem),
    ]
        
#   relative words features (PRE)
    if iw > 0:
        pre_w, pre_lex, pre_pos = retrieve(sent, iw-1)  
        features.extend([
            '-w.len=%s' % len(pre_w),
            '-w.type=%s' % rev_type(pre_w),
            '-w.shape=%s' % w_shape(pre_w),
            '-w.lexem=%s' % pre_lex,
            '-w.pos=%s' % pos,
            '-w.is_wrd=%s' % is_word(pre_lex)
        ])
        
    else:
        features.append('F:BoS')
        
#   relative words features (POST)
    len_sent = len(sent)
    if iw < len_sent - 1:
        post_w, post_lex, post_pos = retrieve(sent, iw+1)
        features.extend([
            '+w.len=%s' % len(post_w),
            '+w.type=%s' % rev_type(post_w),
            '+w.shape=%s' % w_shape(post_w),
            '+w.lexem=%s' % post_lex,
            '+w.pos=%s' % post_pos,
            '+w.is_wrd=%s' % is_word(post_pos)
        ])        

    if iw == len_sent - 2:
        features.append('F:EoS')
    elif iw == len_sent - 1:
        features.append('S:pnt')
#     ToDO test effect
    if iw>1 and iw < len_sent: 
        _, prev2_lex, prev2_pos = retrieve(sent, iw-2)
        features.extend([
            '-2w.lexem=%s' % prev2_lex,
            '-2w.pos=%s' % prev2_pos,
            '-2w.is_wrd=%s' % is_word(prev2_lex)
        ]) 
                
    return features
# get features per sentence
sent_word_features=lambda sent, topic:[word_features(sent, i, topic) for i in range(len(sent))]
# get NER labels per sentence
sent_word_ner=lambda sent: [word_tag.rsplit('_',1)[1] for word_tag, _ in sent]

#test ToDO delete
iter_sen = itertools.islice(iter_sentences(fn_train, lemma=True), 1, 2)
sent_word_features(next(iter_sen), '2')

[['w.len=5',
  'w.type=1',
  'w.iw=0',
  'w.shape=Xxxxx',
  's.topic=2',
  'w.pos=NN',
  'w.is_wrd=True',
  'F:BoS',
  '+w.len=9',
  '+w.type=1',
  '+w.shape=Xxxxxxxxx',
  '+w.lexem=blackburn',
  '+w.pos=NN',
  '+w.is_wrd=False',
  'F:EoS'],
 ['w.len=9',
  'w.type=1',
  'w.iw=1',
  'w.shape=Xxxxxxxxx',
  's.topic=2',
  'w.pos=NN',
  'w.is_wrd=False',
  '-w.len=5',
  '-w.type=1',
  '-w.shape=Xxxxx',
  '-w.lexem=peter',
  '-w.pos=NN',
  '-w.is_wrd=True',
  'S:pnt']]

### Save features to FILE

In [68]:
get_sent_count = lambda fn : sum(1 for _ in iter_sentences_raw(fn))
train_len = get_sent_count(fn_train)
print "TRAIN -", train_len
testa_len = get_sent_count(fn_testa)
print "TESTA -", testa_len
testb_len = get_sent_count(fn_testb)
print "TESTB -", testb_len
print train_len + testa_len +testb_len

TRAIN - 14041
TESTA - 3250
TESTB - 3453
20744


In [69]:
def iter_preserve(fn):
    dt = map(lambda d: d.argmax(), doc_topic)
    if fn == 'train':
        return itertools.izip(iter_sentences(fn_train), itertools.islice(dt, 0, train_len))
    if fn == 'testa':
        frm = train_len
        to = train_len + testa_len
        return itertools.izip(iter_sentences(fn_testa), itertools.islice(dt, frm, to))
    if fn == 'testb':
        frm = train_len + testa_len
        return itertools.izip(iter_sentences(fn_testb), itertools.islice(dt, frm))
        
# iter = iter_preserve('train')
# print sum(1 for _ in iter)
# iter = iter_preserve('testa')
# print sum(1 for _ in iter)
# iter = iter_preserve('testb')
# print sum(1 for _ in iter)

### Reveal topics 

In [58]:
def show_topic(t_id):
    d= []
    frm =0
    size = 500
    for i, s in enumerate(itertools.islice(iter_sentences_raw(fn_train),frm, frm + size)):
        n = doc_topic[i].argmax()
        if n == t_id:
            d.append(s)
    print "Topic %d" % t_id
    for i,x in enumerate(d):
        print i, x
        
show_topic(8)

Topic 8
0 Israel plays down fears of war with Syria .
1 BAGHDAD 1996-08-22
2 THAWRA
3 BEIRUT 1996-08-22
4 AN-NAHAR
5 PARIS 1996-08-22
6 FRONT PAGE
7 Well repairs to lift Heidrun oil output - Statoil .
8 Balance +3,831 +3,428 +12,696 +15,272
9 S. Campbell 69 ; G. Rose 7-73 ) .
10 At Leicester : Leicestershire 343-8 ( P. Simmons 108 , P. Nixon


### Preserve features

In [102]:
%%time
chg_ext = lambda f_name, ext: f_name.rsplit( ".", 1 )[ 0 ] + ext
def features_preserve(fn):
    sent_iter_topic = iter_preserve(fn)
    res = []
    for sent,topic in sent_iter_topic:
        NERs = sent_word_ner(sent)
        features = sent_word_features(sent, topic) # ToDo topic feature add
        for label, crfsuite_features in zip(NERs, features):
            res.append(label + "\t" + "\t".join(crfsuite_features))
        res.append('\n')
    # output file
    f_name = fn + '.input'
    fname = get_full_path(f_name)
    # preserve data
    with open(fname, 'w') as f:
        for row in res:
            f.write(row)
            f.write('\n')
    logging.info("saved to %s" % fname)
            
features_preserve('train')
features_preserve('testa')
features_preserve('testb')

INFO : saved to /home/ildar/nlp/week3/data/train.input
INFO : saved to /home/ildar/nlp/week3/data/testa.input
INFO : saved to /home/ildar/nlp/week3/data/testb.input


CPU times: user 31.3 s, sys: 100 ms, total: 31.4 s
Wall time: 31.5 s


### Classificator

In [110]:
ner_train = "data/train3.input"
ner_testa = "data/testa3.input"
ner_testb = "data/testb3.input"

model = "lbfgs"
fmodel = 'conll2003_%s_2.crfsuite' % model

### remove models

In [72]:
!rm *crfsuite

### list created models

In [ ]:
!ls -l ./*crfsuite

In [111]:
%%time
from subprocess import check_output, call

# algs = (
#     'lbfgs'                # L-BFGS with L1/L2 regularization
#     'l2sgd'                # SGD with L2-regularization
#     'ap'                   # Averaged Perceptron
#     'pa'                   # Passive Aggressive
#     'arow'                 # Adaptive Regularization of Weights (AROW)
#     )

if model == "markov":
    res = call(['crfsuite', 'learn', '-m', fmodel, '-p',  'feature.possible_states=1',ner_train])
elif model == "markov_monfreq2":
    res = call(['crfsuite', 'learn', '-m', fmodel,'-p', 'feature.possible_states=1', 
             '-p', 'feature.minfreq=2', '-p', 'feature.possible_transitions=1', '-p', 'feature.minfreq=2', ner_train])
elif model == "lbfgs":
    res = call(['crfsuite', 'learn', '-m', fmodel, '-a', 'lbfgs', '-p', 'c1=0.1', '-p', 'c2=0.01',
  '-p', 'feature.possible_transitions=1', '-p', 'max_iterations=400', ner_train])
elif model == "lbfgs_strg_bctrg":
    res = call(['crfsuite', 'learn', '-m', fmodel, '-a', 'lbfgs', '-p', 'linesearch=StrongBacktracking', #MoreThuente Backtracking StrongBacktracking
            '-p', 'c1=0.1', '-p', 'c2=0.01', ner_train])
elif model == "l2sgd":
    res = call(['crfsuite', 'learn', '-m', fmodel, '-a', 'l2sgd', '-p', 'c2=0.01', ner_train])
elif model == "ap":
    res = call(['crfsuite', 'learn', '-m', fmodel, '-a', 'ap', '-p', 'max_iterations=150', ner_train])
elif model == "pa":
    #                                                                  0 1 2
    res = call(['crfsuite', 'learn', '-m', fmodel, '-a', 'pa', '-p', 'type=1', '-p', 'c=0.1', '-p', 'max_iterations=300',
            '-p', 'error_sensitive=1', '-p', 'averaging=1', '-p', 'feature.possible_transitions=1', ner_train])
elif model == "arow":
    res = call(['crfsuite', 'learn', '-m', fmodel, '-a', 'arow', '-p', 'max_iterations=200',
            '-p', 'gamma=0.1', '-p', 'variance=0.08', ner_train])

if res == 0:
    out = check_output(['crfsuite', 'dump', fmodel, '|', 'less'])
else:
    print('Error')

CPU times: user 20 ms, sys: 36 ms, total: 56 ms
Wall time: 1min 38s


### Test model

In [112]:
print(fmodel)
out = check_output(['crfsuite', 'tag', '-q', '-m',fmodel, '-t' ,ner_testa])
print(out.decode())

conll2003_lbfgs_2.crfsuite
Performance by label (#match, #model, #ref) (precision, recall, F1):
    B-ORG: (1096, 1252, 1305) (0.8754, 0.8398, 0.8573)
    O: (39760, 40037, 39940) (0.9931, 0.9955, 0.9943)
    B-MISC: (751, 837, 889) (0.8973, 0.8448, 0.8702)
    B-PER: (1622, 1786, 1775) (0.9082, 0.9138, 0.9110)
    I-PER: (1230, 1298, 1279) (0.9476, 0.9617, 0.9546)
    B-LOC: (1660, 1814, 1806) (0.9151, 0.9192, 0.9171)
    I-ORG: (594, 740, 716) (0.8027, 0.8296, 0.8159)
    I-MISC: (247, 300, 327) (0.8233, 0.7554, 0.7879)
    I-LOC: (201, 223, 250) (0.9013, 0.8040, 0.8499)
Macro-average precision, recall, F1: (0.896001, 0.873747, 0.884241)
Item accuracy: 47161 / 48287 (0.9767)
Instance accuracy: 2661 / 3250 (0.8188)
Elapsed time: 0.510000 [sec] (6372.5 [instance/sec])

